In [1]:
#### Text classifcation model - Naive Bayes

In [20]:
import os 
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import islice
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
lemm = WordNetLemmatizer()
 
#import pickle

In [3]:
#os.chdir("/home/visa/Python/News")

In [21]:
df = pd.read_csv("train.csv", header=None)

In [22]:
df.head()

,0,1,2
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [6]:
df.shape

(120000, 3)

In [23]:
df.columns = ["Class","ShortDesc","Desc"]
df.head()

,Class,ShortDesc,Desc
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [7]:
df.shape

(120000, 3)

## Add a full list of stopwords

In [24]:
stops = stopwords.words('english')
ad_stopwords = ['monday', 'b','like', 'com', 'use', 'help', 'create', 'time', 'want', 'a', 'about', 'above', 'across', 'after', 'afterwards',"year","day","month"] 
stops.extend(ad_stopwords)          

In [5]:
new_stops = ['monday',"tuesday","wednesday","thursday","friday","saturday","sunday","yesterday","tomorrow","ap","us","ex"]
stops.extend(new_stops)

In [6]:
stops[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [25]:
#import nltk
def _remove_noise(input_text):
    input_text = str(input_text).encode('ascii', 'ignore')
    input_text = str(input_text).replace(",", "")
   # input_text = str(input_text).replace("\'", "")
   # input_text = str(input_text).replace("\\", "")
   # input_text = str(input_text).replace("-", "")
   # input_text = str(input_text).replace(".", "")
    input_text = re.sub("\d+","", input_text)
    input_text = re.sub("[^a-zA-Z0-9 ]","", input_text)
    words = str(input_text).split()
    noise_free_words = [word for word in words if word.lower() not in stops]
    lower_words = [word.lower() for word in noise_free_words]
    lower_words = [lemm.lemmatize(w) for w in lower_words]
    return lower_words

In [30]:
text = " This is a Test to remove numbers\. 1234\., and special character $"

In [31]:
_remove_noise(text)

['test', 'remove', 'number', 'special', 'character']

In [19]:
a = _remove_noise(text)

In [20]:
a[0]

'test'

In [32]:
df["cleaned"] = df.Desc.apply(_remove_noise)
df.head()

,Class,ShortDesc,Desc,cleaned
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[breuters, shortsellers, wall, street, dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[breuters, private, investment, firm, carlyle,..."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[breuters, soaring, crude, price, plus, worrie..."
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[breuters, authority, halted, oil, exportflows..."
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[bafp, tearaway, world, oil, price, toppling, ..."


In [33]:
#from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
df['lemmed'] = df.cleaned.map(lambda x: ' '.join([lm.lemmatize(y) for y in x]))
df.head()

#port = PorterStemmer()
#df['stemmed'] = df.cleaned.map(lambda x: ' '.join([port.stem(y) for y in x]))



,Class,ShortDesc,Desc,cleaned,lemmed
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[breuters, shortsellers, wall, street, dwindli...",breuters shortsellers wall street dwindlingban...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[breuters, private, investment, firm, carlyle,...",breuters private investment firm carlyle group...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[breuters, soaring, crude, price, plus, worrie...",breuters soaring crude price plus worriesabout...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[breuters, authority, halted, oil, exportflows...",breuters authority halted oil exportflows main...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[bafp, tearaway, world, oil, price, toppling, ...",bafp tearaway world oil price toppling record ...


In [31]:
df.head()

,Class,ShortDesc,Desc,cleaned,lemmed
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...","[breuters, shortsellers, wall, street, dwindli...",breuters shortsellers wall street dwindlingban...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"[breuters, private, investment, firm, carlyle,...",breuters private investment firm carlyle group...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,"[breuters, soaring, crude, price, plus, worrie...",breuters soaring crude price plus worriesabout...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,"[breuters, authority, halted, oil, exportflows...",breuters authority halted oil exportflows main...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","[bafp, tearaway, world, oil, price, toppling, ...",bafp tearaway world oil price toppling record ...


In [34]:
cvec = CountVectorizer(stop_words= stops, min_df=0.001, max_df=0.7)


In [35]:
#from itertools import islice
cvec.fit(df.lemmed)

list(islice(cvec.vocabulary_.items(), 20))


[('breuters', 411),
 ('wall', 2974),
 ('street', 2652),
 ('green', 1255),
 ('private', 2124),
 ('investment', 1455),
 ('firm', 1121),
 ('making', 1679),
 ('play', 2051),
 ('defense', 803),
 ('industry', 1413),
 ('quietly', 2198),
 ('another', 126),
 ('part', 1978),
 ('market', 1703),
 ('soaring', 2551),
 ('crude', 748),
 ('price', 2118),
 ('plus', 2063),
 ('economy', 937)]

In [36]:
len(cvec.vocabulary_)

3073

In [37]:
cvec = CountVectorizer(stop_words=stops, min_df=.001, max_df=.7, ngram_range = (1,2) )
cvec.fit(df.lemmed)
len(cvec.vocabulary_)
#pickle.dump(cvec.vocabulary_,open("feature.pkl","wb"))

3625

In [38]:
cvec_counts = cvec.transform(df.lemmed)
print ('sparse matrix shape:', cvec_counts.shape)
print ('nonzero count:', cvec_counts.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (120000, 3625)
nonzero count: 1810643
sparsity: 0.42%


In [39]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

#transformer = TfidfTransformer()
#loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
#tfidf = transformer.fit_transform(loaded_vec.fit_transform(cvec_counts))

<120000x3625 sparse matrix of type '<class 'numpy.float64'>'
	with 1810643 stored elements in Compressed Sparse Row format>

In [40]:
transformed_weights.shape

(120000, 3625)

In [68]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
332,new york,0.015411
557,united state,0.012301
414,prime minister,0.010010
356,official said,0.009617
361,oil price,0.008131
61,bnew york,0.007770
263,last week,0.007614
260,last night,0.007056
600,york reuters,0.006932
334,new york reuters,0.006725


In [41]:
target = df["Class"]

In [42]:
nb = MultinomialNB(alpha = 50)

In [43]:
nb.fit(transformed_weights, target)

MultinomialNB(alpha=50, class_prior=None, fit_prior=True)

In [46]:
df1 = pd.read_csv("News/test.csv", header=None)

In [47]:
df1.columns = ["Class","ShortDesc","Desc"]
df1.head()

,Class,ShortDesc,Desc
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [48]:
df1.shape

(7600, 3)

In [49]:
df1["cleaned"] = df1.Desc.apply(_remove_noise)
df1.head()

,Class,ShortDesc,Desc,cleaned
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,"[bunion, representing, worker, turner, newall,..."
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...","[bspacecom, toronto, canada, secondteam, rocke..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,"[bap, company, founded, chemistry, researcher,..."
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,"[bap, barely, dawn, mike, fitzpatrick, start, ..."
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,"[bap, southern, california, smogfighting, agen..."


In [50]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df1['lemmed'] = df1.cleaned.map(lambda x: ' '.join([lemmatizer.lemmatize(y) for y in x]))
df1.lemmed.head()

0    bunion representing worker turner newall say d...
1    bspacecom toronto canada secondteam rocketeers...
2    bap company founded chemistry researcher unive...
3    bap barely dawn mike fitzpatrick start shift b...
4    bap southern california smogfighting agency we...
Name: lemmed, dtype: object

In [52]:
cvec1 = CountVectorizer(stop_words= stops,min_df=0.001, max_df=0.7)

In [53]:
#from itertools import islice
cvec1.fit(df1.lemmed)
list(islice(cvec1.vocabulary_.items(), 20))

[('representing', 2379),
 ('worker', 3147),
 ('say', 2479),
 ('talk', 2829),
 ('parent', 2040),
 ('firm', 1169),
 ('federal', 1131),
 ('bspacecom', 468),
 ('toronto', 2919),
 ('canada', 519),
 ('competing', 679),
 ('million', 1818),
 ('ansari', 136),
 ('prize', 2203),
 ('contest', 722),
 ('space', 2668),
 ('flight', 1183),
 ('officially', 1964),
 ('announced', 131),
 ('date', 808)]

In [54]:
cvec1 = CountVectorizer(stop_words=stops, min_df=.005, max_df=.7)
cvec1.fit(df1.lemmed)
len(cvec1.vocabulary_)

707

In [55]:
cvec_counts1 = cvec.transform(df1.lemmed)
print ('sparse matrix shape:', cvec_counts1.shape)
print ('nonzero count:', cvec_counts1.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts1.nnz / (cvec_counts1.shape[0] * cvec_counts1.shape[1])))

sparse matrix shape: (7600, 3625)
nonzero count: 113812
sparsity: 0.41%


In [56]:
transformer1 = TfidfTransformer()
transformed_weights1 = transformer.fit_transform(cvec_counts1)
transformed_weights1

<7600x3625 sparse matrix of type '<class 'numpy.float64'>'
	with 113812 stored elements in Compressed Sparse Row format>

In [57]:
transformed_weights1.shape

(7600, 3625)

In [58]:
preds = nb.predict(transformed_weights1)

In [60]:
confusion_matrix(df1["Class"], preds)

array([[1659,   90,  103,   48],
       [  40, 1816,   21,   23],
       [ 102,   29, 1555,  214],
       [ 111,   64,  216, 1509]], dtype=int64)

In [61]:
accuracy_score(df1["Class"], preds, normalize=True)

0.8603947368421052